In [ ]:
from ray import tune

In [ ]:
from pathlib import Path
from mltrainer import ReportTypes


def train_rnn_with_attention_ray(config):
    import sys
    sys.path.append("/home/azureuser/machinelearning-melissa/networks")
    
    import torch
    from torch import optim
    from ray import tune
    from RNN import RecurrentNeuralNetworkWithAttention, ModelConfig
    from mltrainer import Trainer, TrainerSettings
    from mads_datasets import DatasetFactoryProvider, DatasetType
    from mltrainer.preprocessors import PaddedPreprocessor
    from mltrainer.metrics import Accuracy

    accuracy = Accuracy()
    loss_fn = torch.nn.CrossEntropyLoss()

    # Data
    factory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
    preprocessor = PaddedPreprocessor()
    streamers = factory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
    train = streamers["train"]
    valid = streamers["valid"]

    trainstreamer = train.stream()
    validstreamer = valid.stream()

    # Model
    model_config = ModelConfig(
        input_size=3,
        hidden_size=config["hidden_size"],
        num_layers=config["num_layers"],
        output_size=20,
        dropout=config["dropout"]
    )

    model = RecurrentNeuralNetworkWithAttention(model_config)

    settings = TrainerSettings(
        epochs=10, 
        metrics=[accuracy],
        train_steps=len(train),
        logdir=Path("ray_logs"),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD],
        earlystop_kwargs = {
            "save": False, # save every best model, and restore the best one
            "verbose": True,
            "patience": 5, # number of epochs with no improvement after which training will be stopped
        }
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )

    trainer.loop()
    metrics, test_loss = trainer.evalbatches()
    print("!!!!!!!!!!!!!! DEBUG metrics dict:", metrics)
    tune.report(metrics=metrics)

In [ ]:
search_space = {
    "hidden_size": tune.choice([64, 128, 256, 512]),
    "num_layers": tune.choice([1, 2, 3,4, 5]),
    "dropout": tune.uniform(0.0, 0.5),
    "lr": tune.loguniform(1e-4, 1e-2),
}

In [ ]:
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

scheduler = ASHAScheduler(
    metric="Accuracy",  # of "accuracy" als je dat logt
    mode="max",  # of "max" als je accuracy logt
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(
    metric_columns=["loss", "accuracy", "training_iteration"]
)

result = tune.run(
    train_rnn_with_attention_ray,
    config=search_space,
    num_samples=10,  # aantal combinaties om te testen
    scheduler=scheduler,
    progress_reporter=reporter,
    resources_per_trial={"cpu": 1, "gpu": 0}  # of {"gpu": 1} als je GPU gebruikt
)


In [ ]:
best_result = result.get_best_config(metric="Accuracy", mode="max")

best_trial = result.get_best_trial(metric="Accuracy", mode="max", scope="all")
best_accuracy = best_trial.last_result["Accuracy"]

print(best_result)
print("Beste Accuracy:", best_accuracy)